# Few Shot Learning with Relation net on custom data 

## Preparing

In [1]:
# mount gdrive 

from google.colab import drive 
import os 

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [26]:
# We will clone this repo under this directory 
os.chdir("/content/gdrive/MyDrive/Colab Notebooks/")

In [ ]:
# git clone this repo 
!git clone https://github.

In [27]:
# then change directory to build custom data set 
os.chdir("./FSL_Relationnet_custom_data/custom_data/")

## image tranformation

In [ ]:
# model requires certain square image input  

# but too much error was corrupted during torch.transfrom 
# so I recommend that you should transfrom image using img.resize 

from glob import glob 
from PIL import Image


directr = "/content/gdrive/MyDrive/Colab Notebooks/FSL_Relationnet_custom_data/custom_data/"
forfor = glob(directr+'*')
print(forfor)

for folds in forfor:
  print(folds.split(".")[-1])

for folds in forfor:
    if folds.split(".")[-1] != "json" or "py":
      print("collecting sub_folder root  files")
      
      imgs = glob(folds + "/*")
      print(imgs)

      for f in imgs:
        img = Image.open(f)
        img_resize = img.resize((256, 256), Image.HAMMING)
        img_resize.save(f)
        img.close()
      continue

    if folds.split(".")[-1] == "json" or "py":
      print("excluding json and py files")
      continue


## build custom data file

In [28]:
# write_custom_data.py
# > reads img file and writes json format files
# > From line 50 ~ ( for-loops ) you should change labels and encodings 

!python write_custom_data.py

base -OK
val -OK
novel -OK


In [6]:
# Tree structure of the custom dataset we need

from pathlib import Path

# prefix components:
space =  '    '
branch = '│   '
# pointers:
tee =    '├── '
last =   '└── '


def tree(dir_path: Path, prefix: str=''):
    """A recursive generator, given a directory Path object
    will yield a visual tree structure line by line
    with each line prefixed by the same characters
    """    
    contents = list(dir_path.iterdir())
    # contents each get pointers that are ├── with a final └── :
    pointers = [tee] * (len(contents) - 1) + [last]
    for pointer, path in zip(pointers, contents):
        yield prefix + pointer + path.name
        if path.is_dir(): # extend the prefix and recurse:
            extension = branch if pointer == tee else space 
            # i.e. space because last, └── , above so no more |
            yield from tree(path, prefix=prefix+extension)

In [7]:
# Your dataset structure should be like this  
# if your target folder is not same with current working directory,
# change target_folder_dir 

target_folder_dir = os.getcwd()
for line in tree(Path(target_folder_dir)):
    print(line)

├── class1
│   ├── img.02.png
│   ├── img.01.png
│   └── img.03.png
├── class2
│   ├── img.01.png
│   ├── img.02.png
│   └── img.03.png
├── class3
│   ├── img.02.png
│   ├── img.03.png
│   └── img.01.png
├── base.json
├── val.json
├── novel.json
└── write_custom_data.py


## Run FSL


In [30]:
os.chdir("/content/gdrive/MyDrive/Colab Notebooks/FSL_Relationnet_custom_data")

## Change configuration file 

In [ ]:
# open configs.py and change the first and the last line 

# file is written this way 

#--> save_dir                 = '/content/gdrive/MyDrive/Colab Notebooks/FSL_Relationnet_custom_data'

# data_dir = {}
# data_dir['CUB']             = './filelists/CUB/' 
# data_dir['miniImagenet']    = './filelists/miniImagenet/' 
# data_dir['omniglot']        = './filelists/omniglot/' 
# data_dir['emnist']          = './filelists/emnist/' 
# data_dir['custom']        = '/content/gdrive/MyDrive/Colab Notebooks/FSL_Relationnet_custom_data/custom_data'

#--> data_dir['custom'] = "your custom data set directory "

In [ ]:
# you are prepared to run train.py file 

In [ ]:
#if test_n_way is smaller than train_n_way, 
#reduce n_query to keep batch size small
# test_n_way >= train_n_way
# n-shot < number of data image

# running time is measured 

import time
start = time.time()
print("---------------current start time ---------- : " , start)
!python train.py --dataset "custom" --method relationnet --test_n_way 2 --train_n_way 2 --n_shot 2 --num_classes 3 --start_epoch 0 --stop_epoch 5 --save_freq 2

print("time :", time.time() - start)

In [ ]:
# In this step, you are changing trained model output ( tar file ) into hdf5 file 

# modify save_feature file > 74th Line
# check output directory 

# if you change backbone and method 
# change 74th line 

# if you use different datset 
# change custom data directory in configs.py file 


!python save_features.py --dataset "custom" \
--method relationnet \
--model Conv4 \
--test_n_way 2 \
--train_n_way 2 \
--n_shot 2 \
--save_iter 4

## run test with saved weight

* Only modify test.py file to change options  

* line 25 : change n_query and n_support data set for eval

* line 53 line -> iter num -> how many test we gonna apply 

* line 152 ~ 157 : 
change to load different hdf5 file ( root dirdirdir , Conv type, Network type, method, hwmany , file_type )

* change config file when you change image directory ( using different image set ) 

In [ ]:
!python ./test.py \
--dataset custom \
--method relationnet \
--train_n_way 2 \
--test_n_way 2 \

## checkout saved results

In [ ]:
!xdg-open ./record/results.txt